In [0]:
#!import "../DataModel/DataStructure"
#!import "../Utils/Queries"

# Cashflower

## Identity 
Identity is used in the calculation (performed by Scopes) during import. 

In [0]:
public record CashflowIdentity{
    public string DataNode {get; set;}
    public string AmountType {get; set;}
    public int? AccidentYear {get; set;}
    public Type Type {get; set;}
}

# Storage

The Storage, preload and prepare the data that are required during the calculation. The Storage is consumed by the Scopes. 

# Interface

In [0]:
public interface ICashflowStorage
{   
    protected IDataSource querySource {get;} 
    protected IWorkspace workspace {get;}
    protected Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache {get;}
    protected ImportArgs args {get;}

    public string pillar {get; set; }
    public Cashflow[] CashflowsByIdentity { get; set; } //rename to CashflowDefinitions
    public CashflowIdentity[] Identities {get; set;}
    
    //From ImportStorage
    //Time Periods
    public (int Year, int Month) CurrentReportingPeriod => (args.Year, args.Month);
    public (int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic
    //Partitions
    public Guid PartitionByRn {get; set; }
    public Guid TargetPartition {get; set; }
    public Guid DefaultPartition {get; set; }

    // Initialize
    public async Task BaseInitializeAsync()
    {
        //Get Partitions
        PartitionByRn = (await querySource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        TargetPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        DefaultPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Cashflow Definition
       CashflowsByIdentity = await workspace.QueryCashflowAsync<Cashflow,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, args, pillar);
        
        // Identities
        Identities = CashflowsByIdentity.Select(x => new CashflowIdentity() 
            { 
                DataNode = x.DataNode,
                AmountType = x.AmountType,
                AccidentYear = x.AccidentYear,
                Type = x.GetType(),
            }
        ).Distinct().ToArray();
    }

    protected static async Task DefaultInitializationAsync(ICashflowStorage iCashflowStorage) => await iCashflowStorage.BaseInitializeAsync();
   
    // Getters
    public IEnumerable<Cashflow> GetCashflow<T>(CashflowIdentity identity) where T : Cashflow =>
        CashflowsByIdentity.Where(x => x.GetType() == typeof(T) && x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);

    public string GetPillar() => "Pillar not set";
}

In [0]:
public class CashflowStorage : ICashflowStorage
{
    public IDataSource querySource {get;} 
    public IWorkspace workspace {get;}
    public Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache {get;}
    public ImportArgs args {get;}

    public string pillar {get; set; }
    public Cashflow[] CashflowsByIdentity { get; set; }
    public CashflowIdentity[] Identities {get; set;}
    
    //Partitions
    public Guid PartitionByRn {get; set; }
    public Guid TargetPartition {get; set; }
    public Guid DefaultPartition {get; set; }

    // Constructor
    public CashflowStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }

    public async Task InitializeAsync() { //Can't be called like the method in Interface 
        await ICashflowStorage.DefaultInitializationAsync(this);
    }
}

In [0]:
public class SolvencyStorage : ICashflowStorage
{
    public IDataSource querySource {get;} 
    public IWorkspace workspace {get;}
    public Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache {get;}
    public ImportArgs args {get;}

    public string pillar {get; set; }
    public Cashflow[] CashflowsByIdentity { get; set; }
    public CashflowIdentity[] Identities {get; set;}
    
    //From ImportStorage

    //Partitions
    public Guid PartitionByRn {get; set; }
    public Guid TargetPartition {get; set; }
    public Guid DefaultPartition {get; set; }
    
    // Constructor
    public SolvencyStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }

    public async Task InitializeAsync() { // Can't be called like the method in Interface
        pillar = "Solvency";
        await ICashflowStorage.DefaultInitializationAsync(this);
    }

    //Getters
    public string GetPillar() => pillar;
}

# Class

In [0]:
// public class CashflowStorage
// {   
//     IDataSource querySource; 
//     IWorkspace workspace;
//     Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
//     ImportArgs args;

//     public string pillar = null;
//     public Cashflow[] CashflowsByIdentity { get; set; }
//     public CashflowIdentity[] Identities {get; set;}
    
//     //From ImportStorage
//     //Time Periods
//     public (int Year, int Month) CurrentReportingPeriod => (args.Year, args.Month);
//     public (int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic
//     //Partitions
//     public Guid PartitionByRn;
//     public Guid TargetPartition;
//     public Guid DefaultPartition;


//     // Constructor
//     public CashflowStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
//     {
//         this.querySource = querySource;
//         this.workspace = workspace;
//         hierarchyCache = workspace.ToHierarchicalDimensionCache();
//         this.args = args;
//     }
    
//     // Initialize
//     public async Task InitializeAsync()
//     {
//         //Get Partitions
//         PartitionByRn = (await querySource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
//         TargetPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
//                                                                                                    p.Year == CurrentReportingPeriod.Year &&
//                                                                                                    p.Month == CurrentReportingPeriod.Month &&
//                                                                                                    p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
//         DefaultPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
//                                                                                                     p.Year == CurrentReportingPeriod.Year &&
//                                                                                                     p.Month == CurrentReportingPeriod.Month &&
//                                                                                                     p.Scenario == null).ToArrayAsync()).Single().Id;
//         //Cashflow Definition
//         CashflowsByIdentity = await workspace.QueryCashflowAsync<Cashflow,PartitionByReportingNodeAndPeriod>(querySource, TargetPartition, DefaultPartition, args, pillar);
        
//         // Identities
//         Identities = CashflowsByIdentity.Select(x => new CashflowIdentity() 
//             { 
//                 DataNode = x.DataNode,
//                 AmountType = x.AmountType,
//                 AccidentYear = x.AccidentYear,
//                 Type = x.GetType(),
//             }
//         ).Distinct().ToArray();
//     }
   
//     // Getters
//     public IEnumerable<Cashflow> GetCashflow<T>(CashflowIdentity identity) where T : Cashflow =>
//         CashflowsByIdentity.Where(x => x.GetType() == typeof(T) && x.DataNode == identity.DataNode && x.AmountType == identity.AmountType && x.AccidentYear == identity.AccidentYear);

//     public string GetPillar() => "Pillar not set";
// }

# Solvency

In [0]:
// public class SolvencyStorage : CashflowStorage
// {
//     // Constructor
//     public SolvencyStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace) : base(args, querySource, workspace){}

//     public async Task InitializeAsync() {
//         pillar = "Solvency";
//         await base.InitializeAsync();
//     }

//     //Getters
//     public string GetPillar() => pillar;
// }